In [1]:
from plotly import graph_objects as go
import ipywidgets as widgets
from collections import deque
from copy import deepcopy
import time

In [16]:
def minimaxRoot(depth, board, player):
    global moves
    moves = board.available_moves()
    scores = {}
    bestMove = -9999 if player else 9999

    for move in moves:
        board.push(move)
        value = minimax(depth - 1, board, -10000, 10000, not player)
        scores[move] = value
        board.pop()
        #if value == 1000:
         #   bestMove = value
          #  bestMoveFound = move
           # return bestMoveFound , [],[]
        if player and value >= bestMove:
            bestMove = value
            bestMoveFound = move
        elif not player and value <= bestMove:
            bestMove = value
            bestMoveFound = move
    
    return bestMoveFound , moves,scores
def minimax(depth, board, alpha, beta, player):
    if depth == 0:
        return board.score()

    moves = board.available_moves()
   # if not moves:
    #    return 1000

    if (player):
        bestMove = -9999
        for move in moves:
            board.push(move)
            bestMove = max(bestMove, minimax(depth - 1, board, alpha, beta, not player))
            board.pop()
            alpha = max(alpha, bestMove)
            if beta <= alpha:
                return bestMove
        return bestMove
    else:
        bestMove = 9999
        for move in moves:
            board.push(move);
            bestMove = min(bestMove, minimax(depth - 1, board, alpha, beta, not player))
            board.pop()
            beta = min(beta, bestMove)
            if beta <= alpha:
                return bestMove
        return bestMove

In [17]:
class Board:
    def __init__(self, table, turn=True):
        self.table = deepcopy(table)
        self.queue = []
        self.turn = turn 
        
    def push(self,move):
        self.queue.append(move)
        self.update_table()
        self.turn = not self.turn
        
    def pop(self):
        move = self.queue.pop()
        self.update_table(move)
        self.turn = not self.turn
        return move
        
    def peek(self):
        return self.queue[-1]
    
    def update_table(self,move=None):
        if move != None:
            old, new = move
            self.table[old[0]][old[1]] = not self.turn
            self.table[new[0]][new[1]] = None if old[1]==new[1] else self.turn
        else:
            old, new = self.peek()
            self.table[old[0]][old[1]] = None
            self.table[new[0]][new[1]] = self.turn
            
    def legal_moves(self,x,y):
        x += -1 if self.turn else 1
        moves = []
        for i in range(-1,2):
            if 0 <= x < M and 0 <= y+i < N:
                if self.table[x][y+i] == None and i==0:
                    moves.append((x,y))
                elif self.table[x][y+i] == (not self.turn) and i!=0:
                    moves.append((x,y+i))
        return moves
    
    def available_moves(self):
        moves = []
        for x in range(M):
            for y in range(N):
                if self.table[x][y] == self.turn:
                    for move in self.legal_moves(x,y):
                        moves.append(((x,y),move))
        return moves
    
    def score(self):
        score = 0
        for x in range(M):
            for y in range(N):
                if self.table[x][y] != None:
                    score += M-x if self.table[x][y] == True else -(x+1)
                    if x == 0 and self.table[x][y] == True:
                        return 9999
                    elif x == M-1 and self.table[x][y] == False:
                        return -9999
        score += len([i for row in self.table for i in row if i == True]) - len([i for row in self.table for i in row if i == False])
        return score 

In [18]:
def posible_moves(x,y):
    moves = []
    if TABLE[x][y] == None:
        return moves
    for i in range(-1,2):
        if 0 <= x-1 < M and 0 <= y+i < N:
            if TABLE[x-1][y+i] == None and i==0:
                moves.append((x-1,y))
            elif TABLE[x-1][y+i] == False and i!=0:
                moves.append((x-1,y+i))
    return moves
def game_end(board):
    if False in board.table[M-1]:
        return "Red"
    elif True in board.table[0]:
        return "Blue"
    if not board.available_moves():
        return "Red" if board.turn == True else "Blue"
    return None

In [20]:
debug = True
moves = None
clicked,xf,yf= 0,0,0
def update_fig(trace, points, selector):
    x,y = points.ys[0],points.xs[0]
    global clicked, xf, yf, moves, TABLE
    clicked +=1  
    
    if clicked == 1:
        xf,yf = x,y
        temp = deepcopy(TABLE)
        moves = posible_moves(x,y)
        if not moves:  clicked = 0
            
        for i,j in moves:
            temp[i][j] = 3 if temp[i][j] == False else 2
            
        colors=['#121212' if n == None 
                else'#ff7f0e' if n == 2 else '#1f77b4' if n == 1 
                else '#d62728' if n == 0 else '#2ca02c' for row in temp for n in row]
        fig.data[0].marker['color'] = colors
        
    elif clicked == 2:
        if (x,y) in moves:
            TABLE[x][y] = True
            TABLE[xf][yf] = None
            
        colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
        fig.data[0].marker['color'] = colors
        clicked,moves = 0,None
        
        if game_end(Board(TABLE,False)) != None:
            text.value = f'<font size=4> {game_end(Board(TABLE,False))} wins !!'
            fig.data[0].on_click(None)
            return
        
        time.sleep(0.5) #AI Part
        board = Board(TABLE,False)
        move,moves,scores = minimaxRoot(4, board, False)
        if debug : 
            print([[move,scores[move]] for move in moves])
            
        board.push(move)
        TABLE = board.table
        colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
        fig.data[0].marker['color'] = colors
        
        if game_end(Board(TABLE)) != None:
            text.value = f'<font size=4> {game_end(Board(TABLE))} wins !!'
            fig.data[0].on_click(None)
            return
    
def create_fig():
    fig = go.FigureWidget()
    x = [x for y in range(M) for x in range(N)]
    y = [y for y in range(M) for x in range(N)]
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.add_scatter(x=x, y=y, mode='markers', marker_size=(M+1)*10+N+2, 
        marker_symbol='square', marker_color=colors,hoverinfo='none')
    fig.data[0].on_click(update_fig)
    fig.update_xaxes(range=[-0.5, N - 0.5], dtick=1, side='top', visible=False,autorange=False)
    fig.update_yaxes(range=[-0.5, M - 0.5], dtick=1, autorange='reversed', visible=False)
    fig.update_layout(width=80*N, height=80*N, showlegend=False,margin={'r':0,'l':0,'t':0,'b':0},plot_bgcolor='#212121')

    return fig

N = 4
M = 4
TABLE = [
    [False,]*N,
    [None,None,None,None],
    [None,None,None,None],
    [True,]*N,
]
fig = create_fig()
text = widgets.HTML(value='<font size=4> Hexapawn')
widgets.VBox([fig,text])

    'data': [{'hoverinfo': 'none',
              'marker': {'color': [#d62728, #…

In [21]:
debug = True
def update_fig(trace, points, selector):
    global TABLE
    
    board = Board(TABLE)
    move,moves,scores = minimaxRoot(10, board, True)
    if debug : 
        print([[move,scores[move]] for move in moves])

    board.push(move)
    TABLE = board.table
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.data[0].marker['color'] = colors

    if game_end(Board(TABLE)) != None:
        text.value = f'<font size=4> {game_end(Board(TABLE))} wins !!'
        fig.data[0].on_click(None)
        return 
    
    time.sleep(1) #AI Part
    board = Board(TABLE,False)
    move,moves,scores = minimaxRoot(8, board, False)
    if debug : 
        print([[move,scores[move]] for move in moves])

    board.push(move)
    TABLE = board.table
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.data[0].marker['color'] = colors

    if game_end(Board(TABLE)) != None:
        text.value = f'<font size=4> {game_end(Board(TABLE))} wins !!'
        fig.data[0].on_click(None)
        return
    
def create_fig():
    fig = go.FigureWidget()
    x = [x for y in range(M) for x in range(N)]
    y = [y for y in range(M) for x in range(N)]
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.add_scatter(x=x, y=y, mode='markers', marker_size=(M+1)*10+N+2, 
        marker_symbol='square', marker_color=colors,hoverinfo='none')
    fig.data[0].on_click(update_fig)
    fig.update_xaxes(range=[-0.5, N - 0.5], dtick=1, side='top', visible=False,autorange=False)
    fig.update_yaxes(range=[-0.5, M - 0.5], dtick=1, autorange='reversed', visible=False)
    fig.update_layout(width=80*N, height=80*N, showlegend=False,margin={'r':0,'l':0,'t':0,'b':0},plot_bgcolor='#212121')

    return fig

N = 6
M = 5
TABLE = [
    [False,]*N,
    [None,]*N,
    [None,]*N,
    [None,]*N,
    [True,]*N,
]
fig = create_fig()
text = widgets.HTML(value='<font size=4> Hexapawn')
widgets.VBox([fig,text])

    'data': [{'hoverinfo': 'none',
              'marker': {'color': [#d62728, #…